In [8]:
import pandas as pd
import numpy as np
from toolkit import CustomSpacy

<h2>先用词相似度扩充一下关键字</h2>

In [125]:
contents = pd.read_csv("/Users/starice/Desktop/noun_phrases/type1_2016_1-2-3-4-5-6-7-8-9-10-11-12_nps.csv")

In [126]:
contents.head()

,Unnamed: 0,id,reason_topic,content,content_vector,topic_phrases,topic_phrases_vector,phrase_vectors
0,0,57b031fec2265c7bd4ebc77a,unknown,"经审理查明,2015年7月16日、9月19日,原告在1号店网站的自营店购买NUKA努卡高蛋白...",b'\x91k\xa7?\x05\x15\x87?sp\x99?G\xe9\x04?r\xf...,"['审理', '高蛋白', '营养粉含乳固体饮料', '购物发票', '产品外包装', '本...",NaN,[b'm\xc5\x12@\xcffU>\r\xe0-\xc0\xd8\xf0\x1c\xc...
1,1,57b25887c2265c4d04b3838a,unknown,"本院根据上述认证查明:2015年4月11日,范利军在欧尚超市公司购买捕鱼时光即食三文鱼芥末味...",b'\xc07o?\xa5\xa9I?\xc8[3?\xc2\x12m>\xd4\xc92?...,"['本院根据上述认证', '捕鱼时光', '文鱼芥末味', '时光', '文鱼麻辣味', '...",NaN,[b'\xc4d\xd2?\xac]\x05\xc0\xe6\x91\x99\xbf\xd7...
2,2,57b141dac2265c4d04a4ba20,unknown,"本院认证意见:原告翁艳蕾提供的证据1-6,对其真实性、合法性、关联性本院予以确认,并依据其书...",b'\xda\xef\x8d?\x86\x14\xf5>\xa6\x00&??O\xe4>A...,"['翁艳蕾', '证据', '真实性', '合法性', '确认', '内容', '本案相关事...",NaN,[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0...
3,3,57b23cadc2265c4d04b1e997,unknown,"经庭审质证、辩证,被告对原告举证1无异议。对原告举证2的真实性无异议,原告分两次购买产品,第...",b'\x18\xd1~?\xea]\x0f?J-\xdb>\xee/\xde>#\x11\r...,"['审质证', '辩证', '异议', '产品', '购买', '中文标签', '标签', ...",NaN,[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0...
4,4,5b5b9a43e13823b64b367e32,unknown,"本院经经审理认定事实如下:对于当事人各方没有争议的事实,本院予以确认。当事人双方对原告的起诉...",b'T\xd7~?\xd5u\xae>\x0f\xfa\xd7>K;\xb7>N\xf7\x...,"['本院经经审理', '事实', '争议', '本院', '确认', '当事人', '诉讼时...",NaN,[b'\x9b]\xf4?\x8fHq\xbfA\xf4-\xc0\xf0\x85\xe9>...


In [127]:
cs = CustomSpacy()
cs.global_entity_ruler()
cs.attribute_ruler()
cs.add_money_recognizer()
cs.add_merge_entities()
nlp = cs.nlp

In [128]:
keywords = ['标签', '配料表', '外包装', '虚假', '夸大', '标签格式',
                '名称规范', '产品标准代号', '标识', '假冒产品', '保质期',
                '生产日期', '过期', '原始配料', '原料', '原材料', '非药食同源物质',
                '添加剂', '非食用物质', '食品原料', '商标专用权', '商标违法',
                '商标侵权', '出入境检验', '检疫证明', '生产许可证编号', '生产许可编号',
                '生产许可证', '餐饮', '餐饮食品安全卫生标准', '是否为消费者', '进口食品',
                '尚无国家标准', '未做安全性评估', '产品质检不合格', '质量有问题', '性状']

<h4>将2016年第一类所有的文本都读入spacy中构建corpus</h4>

In [145]:
docs = contents['content'].tolist()
for doc in nlp.pipe(docs):
    pass

In [146]:
ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings["过期"]]]), n=10)
print([nlp.vocab.strings[w] for w in ms[0][0]])

['过期', '过期橄榄油', '过期汤圆', '过期商品', '过期食品', '过期啤酒', '过期食用油销售', '过期标志', '过期产品', '本案过期商品']


<h2>使用LDA找到更多的关键字</h2>

In [147]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import scipy.sparse
from pprint import pprint

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

<h4>Create the Dictinary and Corpus needed for Topic Modeling</h4>

In [148]:
contents['topic_phrases'] = contents['topic_phrases'].apply(lambda r: eval(r))
phrases = contents['topic_phrases'].tolist()

In [149]:
# Create Dictionary
id2word = corpora.Dictionary(phrases)

# Create Corpus
texts = phrases

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [150]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[100:101]]

[[('中华人民共和国食品安全法', 1),
  ('产品', 1),
  ('全国工业产品生产许可证', 1),
  ('判决', 1),
  ('异议', 1),
  ('情况', 1),
  ('支持', 1),
  ('本院', 1),
  ('涉案产品', 1),
  ('证据', 1),
  ('购物发票', 1),
  ('食品', 1),
  ('买卖合同关系', 1),
  ('国家标准', 1),
  ('外包装', 1),
  ('庭审', 1),
  ('植物油', 1),
  ('法律', 1),
  ('法律规定', 1),
  ('白砂糖', 1),
  ('行政法规', 1),
  ('责任', 1),
  ('配料', 1),
  ('食品安全标准', 1),
  ('食品添加剂', 1),
  ('义务', 1),
  ('生产日期', 1),
  ('真实性', 1),
  ('陈述', 1),
  ('中华人民共和国产品质量法', 1),
  ('检查验收制度', 1),
  ('品名', 1),
  ('强制性规定', 1),
  ('描述', 1),
  ('编号', 1),
  ('口感', 1),
  ('不符合', 1),
  ('报告', 1),
  ('食品安全企业标准', 1),
  ('故应', 1),
  ('制造商', 1),
  ('焦点', 1),
  ('糖果', 1),
  ('色剂', 1),
  ('饮品', 1),
  ('企业', 1),
  ('产品执行标准号', 1),
  ('国家食品安全标准', 1),
  ('食品生产许可证号', 1),
  ('产品类别', 1),
  ('产品类型', 1),
  ('产品系属', 1),
  ('产品质量检验所检测报告', 1),
  ('加工备案申请书', 1),
  ('加工备案申请书上', 1),
  ('合理性', 1),
  ('复印签名', 1),
  ('外包装宣传语', 1),
  ('大米制品', 1),
  ('宣传语', 1),
  ('小麦', 1),
  ('小麦粉', 1),
  ('情况表', 1),
  ('植物炭', 1),
  ('植物炭黑', 1),
  ('植物炭黑属', 1),
  ('测试报告底部

In [152]:
# phrases[1:2]

<h4>构建LDA模型</h4>

In [123]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=11, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=200,
                                           alpha='auto',
                                           per_word_topics=True)

In [124]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(11))
# doc_lda = lda_model[corpus]

[(0,
  '0.010*"质证意见" + 0.009*"不利后果" + 0.009*"服务" + 0.008*"食品标签通则" + '
  '0.008*"中华人民共和国侵权责任法" + 0.008*"受害人" + 0.007*"案件事实" + 0.007*"一审法院" + '
  '0.006*"地方标准" + 0.005*"文字"'),
 (1,
  '0.044*"急性" + 0.041*"慢性危害" + 0.038*"营养要求" + 0.035*"食品无毒" + 0.010*"瑕疵" + '
  '0.010*"罚款" + 0.010*"理据" + 0.010*"不安全食品" + 0.008*"概念" + 0.006*"案涉商品"'),
 (2,
  '0.026*"本院" + 0.025*"判决" + 0.024*"食品" + 0.023*"中华人民共和国食品安全法" + 0.022*"支持" + '
  '0.019*"不符合食品安全标准" + 0.018*"销售" + 0.017*"证据" + 0.017*"消费者" + 0.016*"损失"'),
 (3,
  '0.010*"国家" + 0.009*"QS" + 0.008*"许可证" + 0.007*"生产经营" + 0.007*"分支机构" + '
  '0.006*"联系方式" + 0.006*"供货者" + 0.006*"信息" + 0.006*"中文标签" + 0.006*"证明文件"'),
 (4,
  '0.023*"原则" + 0.010*"食品配料" + 0.007*"蔗糖" + 0.007*"生产过程" + 0.006*"食品营养成分基本术语" '
  '+ 0.006*"固体饮料类" + 0.006*"号行政处罚决定书" + 0.005*"涉案产品营养成分表" + 0.005*"燕麦" + '
  '0.005*"氢化"'),
 (5,
  '0.016*"药品" + 0.013*"涉案产品" + 0.012*"食品添加剂" + 0.012*"物质" + 0.012*"生产" + '
  '0.011*"配料" + 0.010*"通知" + 0.010*"普通食品" + 0.010*"卫生证书" + 0.009*"物品"'),
 (6,
  '0.041*"能量" + 0.

<h3>lda在colab上运行</h3>